In [ ]:
import os
from experiment_setup import setups

In [ ]:
# download the dataset, the links can be fund in the README
dataset_path = "./datasets/coveo_ecommerce"
model_path = "./trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [ ]:
params = setups["coveo"]["params_xe"]

In [ ]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [ ]:
def create_keras_gru4rec_script(model_name, train_path, test_path, model_path, loss, optim, layers, batch_size, dropout_p_hidden, learning_rate, n_epochs, m, use_correct_mask_reset):
    s_train_full = f"CUDA_VISIBLE_DEVICES=0 python ./KerasGRU4Rec/model/gru4rec.py --train_path {train_path} --test_path {test_path} --save_path {model_path}/{model_name} --lr {learning_rate} --hidden_size {layers} --dropout_p_hidden {dropout_p_hidden} --batch_size {batch_size}  --optim {optim} --epochs {n_epochs} {'--use_correct_mask_reset' if use_correct_mask_reset else ''}"
    s_test_full = f"CUDA_VISIBLE_DEVICES=0 python /db_vol/aczapp/sandbox/third_parties/KerasGRU4Rec/model/gru4rec.py --train_path {train_path} --test_path {test_path} --resume {model_path}/KerasGRU4Rec/temp/GRU4REC_{n_epochs-1}.h5 --batch_size {batch_size} --eval_only True --m {m}"
    return s_train_full, s_test_full

In [ ]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

### Train the out-of-the-box model

In [ ]:
train_script_oob, test_script_oob = create_keras_gru4rec_script(model_name='keras_gru4rec_oob', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim='adam', layers=100, batch_size=batch_size, dropout_p_hidden=0.25, learning_rate=0.001, n_epochs=n_epochs, m=m, use_correct_mask_reset=False)

In [ ]:
os.system(train_script_oob)

### Test the out-of-the-box model

In [ ]:
os.system(test_script_oob)

## Train & test the minor fix model


In [ ]:
train_script_minor, test_script_minor = create_keras_gru4rec_script(model_name='keras_gru4rec_minorfix', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, use_correct_mask_reset=False)

### Train the minor fix model

In [ ]:
os.system(train_script_minor)

### Test the minor fix model

In [ ]:
os.system(test_script_minor)

## Train & test the major fix model

In [ ]:
train_script_major, test_script_major = create_keras_gru4rec_script(model_name='keras_gru4rec_major', train_path=train_path, model_path=model_path, loss=loss, optim=optim, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, use_correct_mask_reset=True)

### Train the major fix model

In [ ]:
os.system(train_script_major)

### Test the major fix model

In [ ]:
os.system(test_script_major)